Перед нами датасет роста родителей и их детей для 99 семей. 

Цель:
1. Существует ли связь между ростом родителей и ростом детей. 
2. Кто сильнее влияет на рост отец или мать?
3. Есть ли связь между ростом матери и дочки, а также отца и сына?

# **0. Предобработка данных.**





In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sklearn 
import statsmodels.api as sm
import statsmodels
import plotly.express as px
from scipy.stats import mannwhitneyu

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
df = pd.read_csv('/gdrive/MyDrive/test.csv')
df.head(1)

,Family,Father,Mother,Gender,Height,Kids,father,mother,kids
0,1,78.5,67.0,M,73.2,4,199.39,170.18,185.928


Общее описание данных:
* Family -порядковый номер семьи;
* Father - рост отца в ;
*	Mother - рост матери в ;
* Gender - пол ребенка;
*	Height - рост ребенка в ;
* Kids - количество детей в семье;
*	father - рост отца в сантиметрах;
*	mother - рост матери в сантиметрах;
* kids - рост ребенка в сантиметрах.



In [ ]:
df.describe()

,Father,Mother,Height,Kids,father,mother,kids
count,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000
mean,69.232851,64.084410,66.760690,6.135857,175.851441,162.774401,169.572154
std,2.470256,2.307025,3.582918,2.685156,6.274450,5.859844,9.100613
min,62.000000,58.000000,56.000000,1.000000,157.480000,147.320000,142.240000
25%,68.000000,63.000000,64.000000,4.000000,172.720000,160.020000,162.560000
50%,69.000000,64.000000,66.500000,6.000000,175.260000,162.560000,168.910000
75%,71.000000,65.500000,69.700000,8.000000,180.340000,166.370000,177.038000
max,78.500000,70.500000,79.000000,15.000000,199.390000,179.070000,200.660000


1. В данных выбросов и пропуск нет.
2. Среднее значение роста отцов 175.85 с отклонением 6.27. Максимум 199.39, минимум 157.48.
3. Среднее значение роста матерей 162.77 с отклонением 5.86. Максимум 179.07, минимум 147.32.
4. Среднее значение роста детей 169.57 с отклонением 9.1. Максимум 200.66, минимум 142.24.

# **1. Корреляция между ростом родителей и детей.**

Посмотрим на предварительную корреляцию между ростом родителей и ростом детей. 

In [ ]:
df.corr()

,Father,Mother,Height,Kids,father,mother,kids
Father,1.000000,0.073665,0.275355,-0.160023,1.000000,0.073665,0.275355
Mother,0.073665,1.000000,0.201655,-0.020030,0.073665,1.000000,0.201655
Height,0.275355,0.201655,1.000000,-0.126910,0.275355,0.201655,1.000000
Kids,-0.160023,-0.020030,-0.126910,1.000000,-0.160023,-0.020030,-0.126910
father,1.000000,0.073665,0.275355,-0.160023,1.000000,0.073665,0.275355
mother,0.073665,1.000000,0.201655,-0.020030,0.073665,1.000000,0.201655
kids,0.275355,0.201655,1.000000,-0.126910,0.275355,0.201655,1.000000


Видно, что корреляция между ростой детей и отцов составляет 27%  и для матерей 20%.

In [ ]:
father = df['father'].values.reshape(-1, 1)
mother = df['mother'].values.reshape(-1, 1)
kids = df['kids'].values
relatives = np.column_stack((father, mother))

# 1.1 Построим линейную регрессию для значений роста отца и детей.

упростим работу с линейной регрессией и создадим функцию.

In [ ]:
def linear_reg(observed_targets, predicted_targets):
  model = LinearRegression().fit(observed_targets, predicted_targets)
  model.coef_
  model.intercept_
  return model.predict(observed_targets)

Выведем полученные параметры для линейной регрессии:

In [ ]:
kid_f_predicted = linear_reg(father, kids)
res = sm.OLS(kids, sm.add_constant(father)).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     73.51
Date:                Wed, 29 Sep 2021   Prob (F-statistic):           4.35e-17
Time:                        15:31:04   Log-Likelihood:                -3221.4
No. Observations:                 898   AIC:                             6447.
Df Residuals:                     896   BIC:                             6456.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         99.3404      8.197     12.120      0.0

In [ ]:
px.scatter(df, x='father', y='kids', 
                 trendline="ols")

Из рисунка видно, что точки группируются вокруг прямой, однако разброс значений большой. Для большей точности проверим гипотезу, что полученные ряды совпадают.

In [ ]:
stat, p = mannwhitneyu(kids, kid_f_predicted)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=392977.000, p=0.176
Probably the same distribution


# 1.2 Построим линейную регрессию для значений роста матери и детей.

In [ ]:
kid_m_predicted = linear_reg(mother, kids)
res = sm.OLS(kids, sm.add_constant(mother)).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     37.98
Date:                Wed, 29 Sep 2021   Prob (F-statistic):           1.08e-09
Time:                        15:31:39   Log-Likelihood:                -3238.2
No. Observations:                 898   AIC:                             6480.
Df Residuals:                     896   BIC:                             6490.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        118.5945      8.277     14.328      0.0

In [ ]:
px.scatter(df, x='mother', y='kids', 
                 trendline="ols")

Также проверим схожи или отличны ряды данных для предсказанного и имеющегося роста детей.

In [ ]:
stat, p = mannwhitneyu(kids, kid_m_predicted)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=390563.000, p=0.125
Probably the same distribution


Полученная корреляция слабая. Рост отца определяет рост ребенка на 7.5% и рост матери на 4%.

# **2 Построим линейную регрессию для двух параметров.**

In [ ]:
kid_r_predicted = linear_reg(relatives, kids)
res = sm.OLS(kids, sm.add_constant(relatives)).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     54.69
Date:                Wed, 29 Sep 2021   Prob (F-statistic):           3.92e-23
Time:                        15:32:07   Log-Likelihood:                -3205.0
No. Observations:                 898   AIC:                             6416.
Df Residuals:                     895   BIC:                             6430.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         56.6667     10.940      5.180      0.0

Проверим отличие полученных распределений для роста детей.


In [ ]:
stat, p = mannwhitneyu(kid_r_predicted, kids)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=396687.000, p=0.277
Probably the same distribution


Для двухпараметрической линейной регрессии значение квадрата корреляции больше (0.11).Аналогично предыдущим получаем, что полученное распределение похоже на известное распределение роста детей.

# **3.1 Посмотрим на линейную регрессию между ростом отца или матери и дочери.**

In [ ]:
father_d = df[df['Gender'] == 'F']['father'].values.reshape(-1, 1)
mother_d = df[df['Gender'] == 'F']['mother'].values.reshape(-1, 1)
daughter = df[df['Gender'] == 'F']['kids'].values.reshape(-1, 1)
relatives_d = np.column_stack((father_d, mother_d))

Выведем квадраты корреляции для всех вариантов.

In [ ]:
d_f_predicted = linear_reg(father_d, daughter)
d_m_predicted = linear_reg(mother_d, daughter)
d_r_predicted = linear_reg(relatives_d, daughter)


In [ ]:
print('Квадрат кореляции между ростом отца и дочери:',
      sklearn.metrics.r2_score(daughter, d_f_predicted))
print('Квадрат кореляции между ростом матери и дочери:',
      sklearn.metrics.r2_score(daughter, d_m_predicted))
print('Квадрат кореляции между ростом родителей и дочери:',
      sklearn.metrics.r2_score(daughter, d_r_predicted))

Квадрат кореляции между ростом отца и дочери: 0.21046122234250875
Квадрат кореляции между ростом матери и дочери: 0.098406657008278
Квадрат кореляции между ростом родителей и дочери: 0.29754784609831164


Видно, что квадрат корреляции выше для связи между ростом родителей и дочери, чем между кем-то из родителей.

# **3.2 Посмотрим на линейную регрессию между ростом отца или матери и сына.**

Сделаем аналогичные вычисления, но теперь для сыновей.

In [ ]:
father_s = df[df['Gender'] == 'M']['father'].values.reshape(-1, 1)
mother_s = df[df['Gender'] == 'M']['mother'].values.reshape(-1, 1)
son = df[df['Gender'] == 'M']['kids'].values.reshape(-1, 1)
relatives_s = np.column_stack((father_s, mother_s))

In [ ]:
s_f_predicted = linear_reg(father_s, son)
s_m_predicted = linear_reg(mother_s, son)
s_r_predicted = linear_reg(relatives_s, son)


In [ ]:
print('Квадрат кореляции между ростом отца и сына:',
      sklearn.metrics.r2_score(son, s_f_predicted))
print('Квадрат кореляции между ростом матери и сына:',
      sklearn.metrics.r2_score(son, s_m_predicted))
print('Квадрат кореляции между ростом родителей и сына:',
      sklearn.metrics.r2_score(son, s_r_predicted))

Квадрат кореляции между ростом отца и сына: 0.15312927478306992
Квадрат кореляции между ростом матери и сына: 0.1116434600407894
Квадрат кореляции между ростом родителей и сына: 0.23967600206272477


Аналогично с ростом дочерей видно, что квадрат корреляции выше у связи между ростом обоих родителей и роста сына.

# **4. Выводы.**

* Рост родителей на 24% определяет рост сына.
* Рост родителей на 30% определяет рост дочери.
* Рост родителей на 11% определяет рост ребенка.
* Отдельная связь между матерью или отцом и ребенком слаба на этом фоне.